get cifar10 data

In [ ]:
import tensorflow as tf
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


the dataset is too big. we get a smaller one.

In [ ]:
# Create a smaller training set with 500 samples per class
x_train_small = []
y_train_small = []

for i in range(10):
  class_indices = tf.where(y_train == i)[:, 0]
  selected_indices = tf.random.shuffle(class_indices)[:500]
  x_train_small.append(x_train[selected_indices])
  y_train_small.append(y_train[selected_indices])

x_train_small = tf.concat(x_train_small, axis=0)
y_train_small = tf.concat(y_train_small, axis=0)

print("Shape of x_train_small:", x_train_small.shape)
print("Shape of y_train_small:", y_train_small.shape)

Shape of x_train_small: (5000, 32, 32, 3)
Shape of y_train_small: (5000, 1)


In [ ]:
# Create a smaller testing set with 500 samples per class
x_test_small = []
y_test_small = []

for i in range(10):
  class_indices = tf.where(y_test == i)[:, 0]
  selected_indices = tf.random.shuffle(class_indices)[:100]
  x_test_small.append(x_test[selected_indices])
  y_test_small.append(y_test[selected_indices])

x_test_small = tf.concat(x_test_small, axis=0)
y_test_small = tf.concat(y_test_small, axis=0)

print("Shape of x_test_small:", x_test_small.shape)
print("Shape of y_test_small:", y_test_small.shape)

Shape of x_test_small: (1000, 32, 32, 3)
Shape of y_test_small: (1000, 1)


get 40% of data flipped to label 3

In [ ]:
# Create a copy of the training labels
y_train_flipped = y_train_small.numpy().copy()

# Define the percentage of labels to flip
flip_percent = 0.4

# Determine the number of labels to flip
num_labels_to_flip = int(flip_percent * len(y_train_flipped))

# Get the indices of the labels to flip
flip_indices = np.random.choice(len(y_train_flipped), num_labels_to_flip, replace=False)

# Loop over the indices to flip and set them to 3
for idx in flip_indices:
    y_train_flipped[idx] = 3
    

try baseline

In [ ]:
# set random seed
tf.random.set_seed(42)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(128, activation='relu',use_bias=False),
    tf.keras.layers.Dense(10,activation='softmax')
])
# Define the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Define optimizer
optimizer_SGD=tf.keras.optimizers.SGD(learning_rate=0.01)

# Compile the model
model.compile(optimizer=optimizer_SGD, loss=loss_fn, metrics=['accuracy'])

T=100
for t in range(T):
  if (t!=0):
    model.load_weights('model_weights.h5')
  # Train the model
  model.fit(x_train_small, y_train_small,batch_size=500)
  model.save_weights('model_weights.h5')

# Evaluate the model
model.evaluate(x_test_small, y_test_small)

32/32 [==============================] - 0s 3ms/step - loss: 2.3019 - accuracy: 0.0990


[2.3018734455108643, 0.0989999994635582]

In [ ]:
# set random seed
tf.random.set_seed(43)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(128, activation='relu',use_bias=False),
    tf.keras.layers.Dense(10,activation='softmax')
])
# Define the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Define optimizer
optimizer_SGD=tf.keras.optimizers.SGD(learning_rate=0.01)

# Compile the model
model.compile(optimizer=optimizer_SGD, loss=loss_fn, metrics=['accuracy'])

T=100
for t in range(T):
  if (t!=0):
    model.load_weights('model_weights.h5')
  # Train the model
  model.fit(x_train_small, y_train_flipped,batch_size=500)
  model.save_weights('model_weights.h5')

# Evaluate the model
model.evaluate(x_test_small, y_test_small)

32/32 [==============================] - 0s 2ms/step - loss: 2.5500 - accuracy: 0.1000


[2.55001163482666, 0.10000000149011612]

try our method

In [ ]:
# Create a smaller validation set with 10 samples per class
val_indices = np.array([], dtype=np.int32)

# Iterate over the classes and randomly select 10 samples for each class
for class_idx in range(10):
    class_indices = np.where(y_train == class_idx)[0]
    val_class_indices = np.random.choice(class_indices, size=10, replace=False)
    val_indices = np.concatenate([val_indices, val_class_indices])

# Remove the validation samples from the training set
x_val = x_train[val_indices]
y_val = y_train[val_indices]

print("Shape of x_train_small:", x_val.shape)
print("Shape of y_train_small:", y_val.shape)

Shape of x_train_small: (100, 32, 32, 3)
Shape of y_train_small: (100, 1)


sample minibatch

In [ ]:
def SampleMiniBatch(data, batch_size=500):
  x_train, y_train=data
  x_train = x_train.numpy()
  #y_train = y_train.numpy()
  mini_indices = np.array([], dtype=np.int32)

  # Iterate over the classes and randomly select 10 samples for each class
  for class_idx in range(10):
    class_indices = np.where(y_train == class_idx)[0]
    mini_class_indices = np.random.choice(class_indices, size=50, replace=False)
    mini_indices = np.concatenate([mini_indices, mini_class_indices])

  x_mini = x_train[mini_indices]
  y_mini = y_train[mini_indices]
  return [x_mini,y_mini]

In [ ]:
data=(x_train_small,y_train_flipped)
x_mini,y_mini=SampleMiniBatch(data)
print("Shape of x_mini:", x_mini.shape)
print("Shape of y_mini:", y_mini.shape)

Shape of x_mini: (500, 32, 32, 3)
Shape of y_mini: (500, 1)


In [ ]:
# set random seed
tf.random.set_seed(43)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3),name='flatten_layer'),
    tf.keras.layers.Dense(128, activation='relu',use_bias=False,name='dense_layer'),
    tf.keras.layers.Dense(10,activation='softmax')
])
# Define the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Define optimizer
optimizer_SGD=tf.keras.optimizers.SGD(learning_rate=0.01)

# Compile the model
model.compile(optimizer=optimizer_SGD, loss=loss_fn, metrics=['accuracy'])

# Define the model temp
# Define the model
model_temp = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3),name='flatten_layer'),
    tf.keras.layers.Dense(128, activation='relu',use_bias=False,name='dense_layer'),
    tf.keras.layers.Dense(10,activation='softmax')
])
# Define the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Define optimizer
optimizer_SGD=tf.keras.optimizers.SGD(learning_rate=0.01)

# Compile the model
model_temp.compile(optimizer=optimizer_SGD, loss=loss_fn, metrics=['accuracy'])

T=100
for t in range(T):
  print("t=",t)
  data=(x_train_small,y_train_flipped)
  x_mini, y_mini = SampleMiniBatch(data)
  if (t!=0):
    model_temp.load_weights('model_weights.h5')
  # Train the model
  model_temp.fit(x_mini, y_mini)
  # Create a new model that outputs the output of the desired layer
  intermediate_layer_model = tf.keras.models.Model(inputs=model_temp.input,
                                                 outputs=model_temp.get_layer('flatten_layer').output)
  # Get the output of the desired layer for a given input
  z_train = intermediate_layer_model.predict(x_mini)
  z_validation = intermediate_layer_model.predict(x_val)
  z_combine=tf.matmul(z_train,tf.transpose(z_validation))
  comparison_matrix = (y_mini == tf.transpose(y_val))
  integer_matrix = tf.cast(comparison_matrix, dtype=tf.float32)
  weight_mat=tf.multiply(z_combine, integer_matrix)
  weight_ours = tf.reduce_sum(weight_mat, axis=1, keepdims=True)
  weight_ours=weight_ours/np.sum(weight_ours)
  if (t!=0):
    model.load_weights('model_weights.h5')
  model.fit(x_mini, y_mini,sample_weight=weight_ours)
  model.save_weights('model_weights.h5')

# Evaluate the model
model.evaluate(x_test, y_test)

t= 0
16/16 [==============================] - 1s 10ms/step - loss: 0.3701 - accuracy: 0.1120
t= 1
16/16 [==============================] - 0s 5ms/step - loss: 0.1452 - accuracy: 0.1460
t= 2
16/16 [==============================] - 0s 5ms/step - loss: 0.1139 - accuracy: 0.1720
t= 3
16/16 [==============================] - 0s 5ms/step - loss: 0.0915 - accuracy: 0.1680
t= 4
16/16 [==============================] - 0s 4ms/step - loss: 0.0845 - accuracy: 0.1540
t= 5
16/16 [==============================] - 0s 6ms/step - loss: 0.0717 - accuracy: 0.1600
t= 6
16/16 [==============================] - 0s 5ms/step - loss: 0.0424 - accuracy: 0.1800
t= 7
16/16 [==============================] - 0s 5ms/step - loss: 0.0177 - accuracy: 0.1640
t= 8
16/16 [==============================] - 0s 4ms/step - loss: 0.0129 - accuracy: 0.1160
t= 9
16/16 [==============================] - 0s 5ms/step - loss: 0.0106 - accuracy: 0.1260
t= 10
16/16 [==============================] - 0s 8ms/step - loss: 0.0108 - acc

[2.8233845233917236, 0.11230000108480453]